<a href="https://colab.research.google.com/github/Tseng0318/yolo_waste/blob/main/fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.listdir("/content/drive/MyDrive/Zero_Waste")

['zerowaste-f.zip', 'ZeroWaste-Dataset', '__MACOSX']

In [ ]:
import json

# Path to one of the label.json files (train, val, or test)
json_path = "/content/drive/MyDrive/Zero_Waste/train/labels.json"

# Load JSON
with open(json_path, "r") as f:
    data = json.load(f)

# Extract category names
categories = {cat["id"]: cat["name"] for cat in data["categories"]}

# Print class names and count
print(f"✅ Number of classes: {len(categories)}")
print("Classes:", categories)

✅ Number of classes: 4
Classes: {1: 'rigid_plastic', 2: 'cardboard', 3: 'metal', 4: 'soft_plastic'}


In [ ]:
from tqdm import tqdm

# Define dataset path
dataset_path = "/content/drive/MyDrive/Zero_Waste"
splits = ["train", "val", "test"]

# Updated class mapping based on your dataset
category_map = {1: 0, 2: 1, 3: 2, 4: 3}  # COCO category_id → YOLO class_id

# Function to convert COCO JSON to YOLO format
def convert_coco_to_yolo(json_path, images_folder, labels_folder):
    os.makedirs(labels_folder, exist_ok=True)  # Create labels folder if not exists

    # Load COCO JSON file
    with open(json_path, "r") as f:
        data = json.load(f)

    # Convert each annotation
    for annotation in tqdm(data["annotations"], desc=f"Processing {json_path}"):
        image_id = annotation.get("image_id")
        category_id = annotation.get("category_id")
        bbox = annotation.get("bbox", [])  # COCO format: [x, y, width, height]

        # Skip invalid bounding boxes
        if not bbox or category_id not in category_map:
            continue

        # Find corresponding image
        image_info = next((img for img in data["images"] if img["id"] == image_id), None)
        if not image_info:
            continue

        image_width, image_height = image_info["width"], image_info["height"]
        image_filename = image_info["file_name"].replace(".jpg", ".txt")

        # Convert bounding box to YOLO format
        x_center = (bbox[0] + bbox[2] / 2) / image_width
        y_center = (bbox[1] + bbox[3] / 2) / image_height
        width = bbox[2] / image_width
        height = bbox[3] / image_height

        # Get YOLO class ID
        class_id = category_map[category_id]

        # Save YOLO label
        yolo_label = f"{class_id} {x_center} {y_center} {width} {height}\n"
        label_path = os.path.join(labels_folder, image_filename)

        with open(label_path, "a") as f:
            f.write(yolo_label)

# Convert JSON labels for train, val, and test splits
for split in splits:
    json_path = os.path.join(dataset_path, split, "labels.json")  # Correct filename
    images_folder = os.path.join(dataset_path, split, "data")
    labels_folder = os.path.join(dataset_path, split, "labels")  # Save YOLO labels here

    convert_coco_to_yolo(json_path, images_folder, labels_folder)

print("✅ All COCO labels converted to YOLO format and saved in 'labels/' folders!")

Processing /content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/train/labels.json: 100%|██████████| 18002/18002 [01:23<00:00, 216.31it/s]
Processing /content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/val/labels.json: 100%|██████████| 3687/3687 [00:16<00:00, 223.59it/s]
Processing /content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/test/labels.json: 100%|██████████| 5077/5077 [00:26<00:00, 192.66it/s]

✅ All COCO labels converted to YOLO format and saved in 'labels/' folders!


In [ ]:
# List first 10 label files inside the labels folder of the train split
labels_folder = "/content/drive/MyDrive/Zero_Waste/train/labels"
print("Generated YOLO labels:", os.listdir(labels_folder)[:10])  # Show first 10 labels

Generated YOLO labels: ['01_frame_001160.PNG', '01_frame_001170.PNG', '01_frame_001180.PNG', '01_frame_001190.PNG', '01_frame_001200.PNG', '01_frame_001210.PNG', '01_frame_001220.PNG', '01_frame_001230.PNG', '01_frame_001240.PNG', '01_frame_001250.PNG']


In [ ]:
# Define the path to the labels folder
labels_folder = "/content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/train/labels"

# Rename all label files from .PNG to .txt
for filename in os.listdir(labels_folder):
    if filename.endswith(".PNG"):  # Change .PNG to .txt
        old_path = os.path.join(labels_folder, filename)
        new_path = os.path.join(labels_folder, filename.replace(".PNG", ".txt"))
        os.rename(old_path, new_path)

print("✅ All label files renamed to .txt!")

✅ All label files renamed to .txt!


In [ ]:
print("Renamed YOLO labels:", os.listdir(labels_folder)[:10])  # Show first 10 labels

Renamed YOLO labels: ['01_frame_001160.txt', '01_frame_001170.txt', '01_frame_001180.txt', '01_frame_001190.txt', '01_frame_001200.txt', '01_frame_001210.txt', '01_frame_001220.txt', '01_frame_001230.txt', '01_frame_001240.txt', '01_frame_001250.txt']


In [ ]:
test_labels_folder = "/content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/test/labels"

# Check if the folder exists
if os.path.exists(test_labels_folder):
    files = os.listdir(test_labels_folder)
    if len(files) > 0:
        print("✅ Test labels found:", files[:10])  # Show first 10 files
    else:
        print("❌ No test labels found! The folder is empty.")
else:
    print("❌ The test labels folder does not exist.")

✅ Test labels found: ['05_frame_000001.PNG', '05_frame_000011.PNG', '05_frame_000021.PNG', '05_frame_000031.PNG', '05_frame_000041.PNG', '05_frame_000051.PNG', '05_frame_000061.PNG', '05_frame_000071.PNG', '05_frame_000081.PNG', '05_frame_000091.PNG']


In [ ]:
# Rename all test label files from .PNG to .txt
for filename in os.listdir(test_labels_folder):
    if filename.endswith(".PNG"):  # Change .PNG to .txt
        old_path = os.path.join(test_labels_folder, filename)
        new_path = os.path.join(test_labels_folder, filename.replace(".PNG", ".txt"))
        os.rename(old_path, new_path)

print("✅ All test label files renamed to .txt!")

✅ All test label files renamed to .txt!


In [ ]:
val_labels_folder = "/content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/val/labels"

# Check if the folder exists
if os.path.exists(val_labels_folder):
    files = os.listdir(val_labels_folder)
    if len(files) > 0:
        print("✅ Validation labels found:", files[:10])  # Show first 10 files
    else:
        print("❌ No validation labels found! The folder is empty.")
else:
    print("❌ The validation labels folder does not exist.")

In [ ]:
# Define the path to the validation labels folder
val_labels_folder = "/content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/val/labels"

# Rename all validation label files from .PNG to .txt
for filename in os.listdir(val_labels_folder):
    if filename.endswith(".PNG"):  # Change .PNG to .txt
        old_path = os.path.join(val_labels_folder, filename)
        new_path = os.path.join(val_labels_folder, filename.replace(".PNG", ".txt"))
        os.rename(old_path, new_path)

print("✅ All validation label files renamed to .txt!")

In [ ]:
print("Renamed Validation Labels:", os.listdir(val_labels_folder)[:10])  # Show first 10 labels

## Train on YOLO

In [ ]:
yaml_path = "/content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/garbage.yaml"

yaml_content = """train: /content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/train/data
val: /content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/val/data
test: /content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/test/data

nc: 4
names: ['rigid_plastic', 'cardboard', 'metal', 'soft_plastic']
"""

with open(yaml_path, "w") as f:
    f.write(yaml_content)

print("✅ garbage.yaml file created successfully!")

✅ garbage.yaml file created successfully!


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.3/915.3 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
val_labels_folder = "/content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/val/labels"

# Check if the folder exists
if os.path.exists(val_labels_folder):
    files = os.listdir(val_labels_folder)
    if len(files) > 0:
        print("✅ Validation labels found:", files[:10])  # Show first 10 files
    else:
        print("❌ No validation labels found! The folder is empty.")
else:
    print("❌ The validation labels folder does not exist.")

✅ Validation labels found: ['04_frame_000001.PNG', '04_frame_000011.PNG', '04_frame_000021.PNG', '04_frame_000031.PNG', '04_frame_000041.PNG', '04_frame_000051.PNG', '04_frame_000061.PNG', '04_frame_000071.PNG', '04_frame_000081.PNG', '04_frame_000091.PNG']


In [ ]:
import os

# Define the path to the validation labels folder
val_labels_folder = "/content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/val/labels"

# Rename all validation label files from .PNG to .txt
for filename in os.listdir(val_labels_folder):
    if filename.endswith(".PNG"):  # Change .PNG to .txt
        old_path = os.path.join(val_labels_folder, filename)
        new_path = os.path.join(val_labels_folder, filename.replace(".PNG", ".txt"))
        os.rename(old_path, new_path)

print("✅ All validation label files renamed to .txt!")

✅ All validation label files renamed to .txt!


In [ ]:
print("Renamed Validation Labels:", os.listdir(val_labels_folder)[:10])  # Show first 10 labels

Renamed Validation Labels: ['04_frame_000001.txt', '04_frame_000011.txt', '04_frame_000021.txt', '04_frame_000031.txt', '04_frame_000041.txt', '04_frame_000051.txt', '04_frame_000061.txt', '04_frame_000071.txt', '04_frame_000081.txt', '04_frame_000091.txt']


In [ ]:
from ultralytics import YOLO

# Load YOLOv10 model
model = YOLO("yolov10n.pt")  # YOLOv10-Nano for fast training

# Train with your dataset
results = model.train(
    data="/content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/garbage.yaml",
    epochs=50,  # Increase if needed
    imgsz=640,  # Image size
    batch=16,  # Adjust based on Colab memory
    device="cuda"  # Use GPU
)

# Save trained model
model.export(format="pt")

Ultralytics 8.3.77 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov10n.pt, data=/content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/garbage.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show

train: Scanning /content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/train/data.cache... 0 images, 3002 backgrounds, 0 corrupt: 100%|██████████| 3002/3002 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /content/drive/MyDrive/Zero_Waste/ZeroWaste-Dataset/train/data.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



Exception ignored in: 